# Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
### misc stuffs
import spacy
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
### torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
%%capture

UsageError: Line magic function `%%capture` not found.


In [3]:
#must print /content, assuming your structure is content / drive/ myDrive/ 142a colab/ (test or train csv's)
test = pd.read_csv('drive/MyDrive/142a colab/test.csv', usecols=[0,2], names = ['posRating', 'text'], nrows= 3000) #rows can be changed to increase time taken for preprocessing
train = pd.read_csv('drive/MyDrive/142a colab/train.csv', usecols = [0, 2], names = ['posRating', 'text'], nrows = 12000)
train.posRating -= 1
test.posRating -= 1
train, val = train_test_split(train, test_size = 0.1, random_state= 69)

nlp = spacy.load('en_core_web_sm')

# GRU Model

In [4]:
def tokenizer(df):
    docs = nlp.pipe(df['text'].str.lower(), batch_size=50, n_process=-1)
    return [[token.text for token in doc if not token.is_punct and not token.is_space and not token.is_stop] for doc in docs]

def padTrunc(tokenized_docs, max_length= 30, padToken = '<PAD>'):
    num_docs = len(tokenized_docs)
    result = np.full((num_docs, max_length), padToken, dtype=object)  # Use `object` for string tokens

    for i, doc in enumerate(tokenized_docs):

        truncated = doc[:max_length]  # Truncate to max_length
        result[i, :len(truncated)] = truncated  # Place truncated tokens into the array

    return result

def embed():
    embeddings = {}
    with open('drive/MyDrive/142a colab/glove.6B.50d.txt', "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            embeddings[word] = vector
    return embeddings

def tokenToEmbedding(tokenized_docs, glove_dict, embedding_dim):
    """
    Convert tokenized documents into embedding vectors.

    Args:
    - tokenized_docs (list of lists): Tokenized and padded documents.
    - glove_dict (dict): Pre-loaded GloVe dictionary mapping words to embeddings.
    - embedding_dim (int): Dimension of GloVe embeddings.

    Returns:
    - np.ndarray: 3D array of embeddings (num_docs, max_length, embedding_dim).
    """
    # Create a 3D array to hold embeddings
    num_docs = len(tokenized_docs)
    max_length = len(tokenized_docs[0])  # Assume all documents are padded to the same length
    embeddings = np.zeros((num_docs, max_length, embedding_dim), dtype=np.float32)

    for i, doc in enumerate(tokenized_docs):
        for j, token in enumerate(doc):
            if token in glove_dict:
                embeddings[i, j] = glove_dict[token]
            else:
                embeddings[i, j] = np.zeros(embedding_dim)  # Use zero vector for OOV or <PAD>

    return embeddings


In [5]:
embeddings = embed()

In [6]:
def preProcess(df):
    mat = padTrunc(tokenizer(df))
    Emb = torch.tensor(tokenToEmbedding(mat, embeddings, 50))
    Lbl = torch.tensor(df['posRating'].values).float()
    return TensorDataset(Emb, Lbl)

In [7]:
#MAX_SEQ_LEN = 30      fixed sequence length as 30, close to mean length
#EMBEDDING_DIM = 50    dimension of GloVe word embeddings

# Initialize DataLoaders
trainMatrix = preProcess(train)
valMatrix = preProcess(val)
train_loader = DataLoader(trainMatrix, batch_size = 27, shuffle = True)
val_loader = DataLoader(valMatrix, batch_size = 20, shuffle = False)

In [8]:
class GRUSentiment(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, drop):
        super(GRUSentiment, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout = drop)
        self.output_layer = nn.Linear(hidden_size, 1)


    def forward(self, x):
        gru_out, h_n = self.gru(x)  # gru_out: (batch, sequence length (30) , hidden_size) H_n: (numLayers, batch_size, hidden_size)
        output = self.output_layer(h_n[-1]).flatten() # last hidden state: (batch, hidden) --> h_n[-1] yields last hidden state, flatten converts shape from 45,1 to 45.
        return output # (batch)

def get_loss_fn():
    return nn.BCEWithLogitsLoss()

def get_validation_accuracy(model, device):
    model.eval()
    correct = 0
    totalPreds = 0
    yPreds = []
    with torch.no_grad():
        for batch in val_loader:
            x = batch[0].to(device)
            y = batch[1].to(device)
            probs = torch.sigmoid(model.forward(x))
            preds = (probs >= 0.5).float()
            yPreds.extend(preds.cpu().numpy())
            correct += (preds == y).sum().item()
            totalPreds += preds.shape[0]
    return (correct / totalPreds), f1_score(val['posRating'].values, yPreds)

def train_model(model,device):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00015)
    criterion = get_loss_fn()

    for epoch in range(20):
        model.train()

        for batch in train_loader:
            x = batch[0].to(device)
            y = batch[1].to(device).float()
            optimizer.zero_grad()
            logits = model.forward(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        val_accuracy = get_validation_accuracy(model,device)
        print(f"Epoch {epoch} Validation Accuracy = {val_accuracy[0]:.4f}")
        print(f"F1 Score = {val_accuracy[1]:.4f}")

    return



In [ ]:
"""
Model hyperparameters & Training
"""
input_size = 50
hidden_size = 150 # Increased hidden size for better representation
num_layers = 2  # Reduced number of layers to prevent overfitting
drop = 0.1 #model parameter dropout probability
model = GRUSentiment(input_size, hidden_size, num_layers, drop)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model, device)

In [ ]:
test_loader = DataLoader(preProcess(test), batch_size = 20, shuffle = False)

In [ ]:
def predict(model, device, test_loader):
    model.eval()
    all_predictions = []

    with torch.no_grad():
        for batch in test_loader:
            x = batch[0].to(device)
            logits = model(x)  # Raw logits
            probs = torch.sigmoid(logits)  # Convert logits to probabilities
            preds = (probs >= 0.5).float()  # Binary predictions
            all_predictions.extend(preds.cpu().numpy())

    return all_predictions

In [ ]:
y_pred = predict(model, device, test_loader)
y_true = test.posRating
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1-Score: {f1:.4f}")
print(f"Test ROC-AUC: {roc_auc:.4f}")

# Multinomial Naive Bayes Model


In [ ]:
import pandas as pd

train = pd.read_csv('drive/MyDrive/142a colab/train.csv', header=None)
test = pd.read_csv('drive/MyDrive/142a colab/test.csv', header=None)
train.columns = ['posRating', 'first_review', 'second_review']
test.columns = ['posRating', 'first_review', 'second_review']

In [ ]:


# Melt the train DataFrame
train_long = pd.melt(
    train,
    id_vars='posRating',
    value_vars=['first_review', 'second_review'],
    var_name='original_column',
    value_name='text'
)
train_long.drop(columns='original_column', inplace=True)

# Melt the test DataFrame
test_long = pd.melt(
    test,
    id_vars='posRating',
    value_vars=['first_review', 'second_review'],
    var_name='original_column',
    value_name='text'
)
test_long.drop(columns='original_column', inplace=True)

# Map the posRating values: 1 -> 0 (negative), 2 -> 1 (positive)
rating_map = {1: 0, 2: 1}
train_long['posRating'] = train_long['posRating'].replace(rating_map)
test_long['posRating'] = test_long['posRating'].replace(rating_map)
train_long.dropna(subset=['posRating', 'text'], inplace=True)
test_long.dropna(subset=['posRating', 'text'], inplace=True)




In [ ]:
#Sam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Extract training and test data
train_texts = train_long['text']
train_labels = train_long['posRating']

test_texts = test_long['text']
test_labels = test_long['posRating']

# Vectorize text (bag-of-words representation)
vectorizer = TfidfVectorizer(stop_words='english')  # limit vocabulary for efficiency
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# Initialize and train the Multinomial Naive Bayes model
mnb_model = MultinomialNB()
mnb_model.fit(X_train, train_labels)

# Predict on the test set
y_pred = mnb_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(test_labels, y_pred))
print("Classification Report:")
print(classification_report(test_labels, y_pred))



# Logistic Regression Model



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print(os.getcwd())

train = pd.read_csv('drive/MyDrive/142a colab/train.csv', header=None)
test = pd.read_csv('drive/MyDrive/142a colab/test.csv', header=None)

train.columns = ['posRating', 'subject', 'text']
test.columns = ['posRating', 'subject', 'text']

train.head()

In [ ]:
x_train = train['text']
y_train = train['posRating']

x_test = test['text']
y_test = test['posRating']

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)


In [ ]:
X_train_tfidf = vectorizer.fit_transform(x_train)
X_test_tfidf = vectorizer.transform(x_test)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

FastText Method

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import FastText
import os
print(os.getcwd())

In [ ]:
train = pd.read_csv('drive/MyDrive/142a colab/train.csv', header=None)
test = pd.read_csv('drive/MyDrive/142a colab/test.csv', header=None)

train.columns = ['posRating', 'subject', 'text']
test.columns = ['posRating', 'subject', 'text']

In [ ]:
X_train = train['text'].astype(str).apply(lambda x: x.lower().split())
y_train = train['posRating']

X_test = test['text'].astype(str).apply(lambda x: x.lower().split())
y_test = test['posRating']

In [ ]:
fasttext_model = FastText(sentences=X_train, vector_size=100, window=5, min_count=5, workers=4)

def get_average_embedding(tokens, model):
    valid_tokens = [t for t in tokens if t in model.wv]
    if not valid_tokens:
        return np.zeros(model.wv.vector_size)
    return np.mean([model.wv[t] for t in valid_tokens], axis=0)


In [ ]:
X_train_emb = np.vstack(X_train.apply(lambda x: get_average_embedding(x, fasttext_model)))
X_test_emb = np.vstack(X_test.apply(lambda x: get_average_embedding(x, fasttext_model)))

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_emb, y_train)

In [ ]:
y_pred = model.predict(X_test_emb)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy (FastText + LR):", acc)

# Conv1D CNN Model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

punc = string.punctuation

# 读取数据
test = pd.read_csv('drive/MyDrive/142a colab/test.csv', names=['posRating', 'subject', 'text'])
train = pd.read_csv('drive/MyDrive/142a colab/train.csv', names=['posRating', 'subject', 'text'])

# 数据清洗
translation_table = str.maketrans("", "", punc)
for df in [train, test]:
    df['subject'] = df['subject'].astype(str).str.translate(translation_table).str.lower().apply(word_tokenize)
    df['text'] = df['text'].astype(str).str.translate(translation_table).str.lower().apply(word_tokenize)

train.posRating -= 1
test.posRating -= 1

# 划分训练集和验证集
train, val = train_test_split(train, test_size=0.005)


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 合并subject和text列
train['combined'] = train['subject'] + train['text']
val['combined'] = val['subject'] + val['text']
test['combined'] = test['subject'] + test['text']

# 创建分词器
tokenizer = Tokenizer(num_words=10000)  # 设置最大词汇数量
tokenizer.fit_on_texts(train['combined'])

# 将文本转化为序列
X_train = tokenizer.texts_to_sequences(train['combined'])
X_val = tokenizer.texts_to_sequences(val['combined'])
X_test = tokenizer.texts_to_sequences(test['combined'])

# 填充序列到固定长度
maxlen = 100  # 设置最大序列长度
X_train = pad_sequences(X_train, maxlen=maxlen, padding='post')
X_val = pad_sequences(X_val, maxlen=maxlen, padding='post')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='post')

# 标签
y_train = np.array(train['posRating'])
y_val = np.array(val['posRating'])
y_test = np.array(test['posRating'])


In [ ]:
# 从 Google Drive 加载 GloVe 文件
glove_path = 'drive/MyDrive/142a colab/glove.6B.50d.txt'

# 加载 GloVe 词向量
embeddings_index = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = vector

print(f"Loaded {len(embeddings_index)} word vectors.")

# 获取 GloVe 词汇表的大小和嵌入维度
embedding_dim = 50  # 使用 50 维的 GloVe 词向量

# 创建一个嵌入矩阵
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

# 为每个词填充嵌入矩阵
for word, i in tokenizer.word_index.items():
    if word in embeddings_index:
        embedding_matrix[i] = embeddings_index[word]
    else:
        embedding_matrix[i] = np.random.randn(embedding_dim)

print(f"Embedding matrix shape: {embedding_matrix.shape}")



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout


# 构建模型，使用预训练的 GloVe 嵌入
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1,
              output_dim=embedding_dim,
              weights=[embedding_matrix],  # 使用 GloVe 词向量初始化嵌入层
              input_length=maxlen,
              trainable=False),  # 如果想要冻结词向量，设置为 False，或者 True 如果你希望训练嵌入
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 输出层，用于二分类
])

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 查看模型概述
model.summary()

# 训练模型
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=512,
    validation_data=(X_val, y_val)
)

# 在测试集上评估模型
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# 保存模型
model.save('sentiment_model.h5')

# 加载模型
from tensorflow.keras.models import load_model
model = load_model('sentiment_model.h5')
